In [1]:

# --- #
# modules

import pandas as pd
import requests
import json
import config
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import os
import datetime

# --- #
# pandas settings

# pd.set_option('display.max_columns',    None)
# pd.set_option('display.max_rows',       200)
# pd.set_option('display.width',          500)
# pd.set_option('display.max_seq_items',  None)
# pd.set_option('display.max_colwidth',   50)
# pd.set_option('expand_frame_repr',      True)

# --- #


In [2]:

# --- #
# achieve token
# --- #

def achieve_token():
    pass

url_token = config.url_token

# data payload

data = {
         'grant_type'   : config.grant_type
        ,'companyId'    : config.companyId
        ,'username'     : config.user_name
        ,'password'     : config.password
        ,'client_id'    : config.client_id
}

try:
    response = requests.post(url_token, data = data)
    # console.print('great success', style = 'good')
    # print('great success')

except:
    # console.print('error', style = 'bad')
    print('error')
    # pass

data    = json.loads(response.content)

token   = data['access_token']

# --- #


In [3]:

# --- #
# get report

head        = {'Authorization': f'Bearer {token}'}

url_report  = f'https://us63-services.dayforcehcm.com/Api/{config.companyId}/V1/Reports/{config.report1}?exampleParameterGuid1=string&exampleParameterGuid2=string&exampleParameterGuid3=string'

response    = requests.get(url_report, headers = head)

data        = json.loads(response.content)

# --- #


In [4]:

# --- #
# extract only the records / data portion of the response

data_filtered   = data['Data']['Rows']

df              = pd.DataFrame.from_dict(data_filtered)

# print(df)

# --- #


In [5]:

# --- #
# convert datetime columns to datetime

df[['EmployeeTAFWCustom_TimeStart', 'EmployeeTAFWCustom_TimeEnd', 'EmployeeTAFWCustom_LastModifiedTimestamp', 'EmployeeTAFWCustom_TimeRequested', 'EmployeeTAFWCustom_TimeStatusChanged']] = df[['EmployeeTAFWCustom_TimeStart', 'EmployeeTAFWCustom_TimeEnd', 'EmployeeTAFWCustom_LastModifiedTimestamp', 'EmployeeTAFWCustom_TimeRequested', 'EmployeeTAFWCustom_TimeStatusChanged']].apply(pd.to_datetime)

# modify format to have bigquery recognise the object as a DATE, python doesn't have a DATE datatype (only datetime)

df['EmployeeTAFWCustom_NetHours'] = df['EmployeeTAFWCustom_NetHours'].astype(float)


# --- #


In [6]:

# --- #
# rename columns



df = df.rename(

    columns = {

             'EmployeeEmploymentStatus_EmployeeNumber'      : 'EmployeeId'
            ,'TAFWStatus_ShortName'                         : 'Status'
            ,'PayAdjCode_ShortName'                         : 'Type'
            ,'EmployeeTAFWCustom_TimeStart'                 : 'DateTimeStart'
            ,'EmployeeTAFWCustom_TimeEnd'                   : 'DateTimeEnd'
            ,'EmployeeTAFWCustom_LastModifiedTimestamp'     : 'DateTimeLastModified'
            ,'EmployeeTAFWCustom_TimeRequested'             : 'DateTimeRequested'
            ,'EmployeeTAFWCustom_TimeStatusChanged'         : 'DateTimeStatusChanged'
            ,'EmployeeTAFWCustom_NetHours'                  : 'HoursQty'

    }
        # ,inplace = True

)

# --- #


In [7]:

# ---#
# loc = len(df.columns) => count number of columns

df.insert(loc = len(df.columns), column = 'DateTimeInserted', value = datetime.datetime.utcnow())

# --- #


In [8]:

# --- #
# connect to bigquery

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.bigquery_credentials_path

client = bigquery.Client()

# --- #


In [9]:

schema = [

    {
         'name'          : 'EmployeeId'
        ,'type'          : 'STRING'
        ,'mode'          : 'REQUIRED'
        ,'description'   : 'dayforce employee id'
    },
    {
         'name'          : 'Status'
        ,'type'          : 'STRING'
        ,'mode'          : 'NULLABLE'
        ,'description'   : 'status of the tafw request'
    },
    {
         'name'          : 'Type'
        ,'type'          : 'STRING'
        ,'mode'          : 'NULLABLE'
        ,'description'   : 'type of tafw (ex. vacation, sick)'
    },
    {
         'name'          : 'DateTimeStart'
        ,'type'          : 'DATETIME'
        ,'mode'          : 'NULLABLE'
        ,'description'   : 'datetime the tafw starts'
    },
    {
         'name'          : 'DateTimeEnd'
        ,'type'          : 'DATETIME'
        ,'mode'          : 'NULLABLE'
        ,'description'   : 'datetime the tafw ends'
    },
    {
         'name'          : 'DateTimeLastModified'
        ,'type'          : 'DATETIME'
        ,'mode'          : 'NULLABLE'
        ,'description'   : 'datetime the record was modified'
    },
    {
         'name'          : 'DateTimeRequested'
        ,'type'          : 'DATETIME'
        ,'mode'          : 'NULLABLE'
        ,'description'   : 'datetime the record was modified'
    },
    {
         'name'          : 'DateTimeStatusChanged'
        ,'type'          : 'DATETIME'
        ,'mode'          : 'NULLABLE'
        ,'description'   : 'datetime the status changed'
    },
    {
         'name'          : 'HoursQty'
        ,'type'          : 'FLOAT'
        ,'mode'          : 'NULLABLE'
        ,'description'   : 'total hours the employee will be away'
    },
    {
         'name'          : 'DateTimeInserted'
        ,'type'          : 'DATETIME'
        ,'mode'          : 'REQUIRED'
        ,'description'   : 'datetime when the record was INSERTed into the warehouse'
    }

]

table_path = f'{config.project_id}.{config.dataset}.stage_{config.table1}'
truncate_table = (f'''TRUNCATE TABLE {table_path}''')

# try:
#     client.get_table(table = table_path)
# # client.delete_table(table_path)
#     client.query(truncate_table)

# except:
#     table = bigquery.Table(table_path, schema = schema)
#     client.create_table(table)

client.query(truncate_table).result()



In [10]:

# ---#

insert = client.load_table_from_dataframe(df, table_path).result()

# ---#


In [11]:

# --- #
# call stored procedure to MERGE between stage table and target

# stored_procedure = (f'''CALL {config.project_id}.{config.dataset}.merge_{config.table1}() ''')

# stored_procedure = (f'CALL `bowery-dev-354722.dayforce.merge_analytics_tafw`()')
stored_procedure = (f'CALL `{config.project_id}.{config.dataset}.merge_{config.table1}`()')

client.query(stored_procedure).result()

# --- #


In [12]:

# --- #
# delete the stage table

# truncate_table = (f'''TRUNCATE TABLE {table_path}''')
# client.delete_table(table_path)
client.query(truncate_table).result()

# --- #


In [13]:

# --- #
# close connection

client.close()

# --- #
